In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "admin"
password = "admin"
shelter = AnimalShelter(username, password)
# class read method must support return of cursor object 
cursor = shelter.read({})

df = pd.DataFrame.from_records(cursor)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H1('Joshua Flores')),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dcc.RadioItems(id='filter-type',
                    options=[{'label':'Water Rescue','value':'WRC'},
                             {'label':'Mountain or Wilderness Rescue', 'value':'MWRC'}, 
                             {'label':'Disaster Rescue or Individual Tracking','value':'DRIT'},
                             {'label':'RESET','value':'Reset'}
                            ], 
                    value = 'Reset'
                  ),
    dt.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
            page_size=10
        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        
        if filter_type=='WRC':
            filter_criteria = ['Labrador Retriever', 'Chesa Bay Retr', 'Newfoundland']
            df1 = df[(df['breed'].str.contains('|'.join(filter_criteria))) &
                     (df['age_upon_outcome_in_weeks'] >= 26) & 
                     (df['age_upon_outcome_in_weeks'] <= 156)& 
                     (df['sex_upon_outcome'] == 'Intact Female')]
            data = df1.to_dict('records')
        elif filter_type=='MWRC':
            filter_criteria = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog','Siberian Husky','Rottweiler']
            df1 = df[(df['breed'].str.contains('|'.join(filter_criteria))) &
                     (df['age_upon_outcome_in_weeks'] >= 26) & 
                     (df['age_upon_outcome_in_weeks'] <= 156)& 
                     (df['sex_upon_outcome'] == 'Intact Male')]
            
            data = df1.to_dict('records')
            
        elif filter_type=='DRIT':
            filter_criteria = ['German Shepherd', 'Doberman Pinsch', 'Rottweiler','Golden Retriever','Bloodhound']
            df1 = df[(df['breed'].str.contains('|'.join(filter_criteria))) &
                     (df['age_upon_outcome_in_weeks'] >= 20) & 
                     (df['age_upon_outcome_in_weeks'] <= 300)& 
                     (df['sex_upon_outcome'] == 'Intact Male')]
            
            data = df1.to_dict('records')
        else:
            data=df.to_dict('records')
        
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
       dcc.Graph(
           figure = px.pie(data_frame=dff,names=dff['breed'], title='breeds'),
       )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app

